```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/basics.ipynb>`, or a {download}`python script <converted/basics.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial. Alternatively, you could run the Python script in a terminal with ``python3 basics.py`` from the folder where the file is located.
```

# Tutorial 4: Activation Maps

This tutorial will discuss how to compute local activation times and activation maps from cardiac optical mapping data using ``optimap``. Local activation times (often referred to as LATs) are times specified in frames or milliseconds at which the tissue became electrically activated. Computing local activation times corresponds to determining when the optical signal in a given pixel passes a certain pre-defined threshold or intensity value. For instance, if the optical trace is normalized and fluctuates betwen [0,1] and an action potential darkens the image (this is the convention that we use in Tutorials 1 and 2), then the tissue could be defined as being electrically 'activated' when the time-series goes below 0.5. We will discuss several examples in this tutorial.

First, we load and preprocess an example dataset in which a planar action potential wave propagates across the ventricles of a rabbit heart:

In [ ]:
import optimap as om

filename = om.utils.retrieve_example_data('Example_01_Sinus_Rabbit_Basler.npy')
video = om.load_video(filename)
video = om.video.rotate_left(video)
video_warped = om.motion.motion_compensate(video, 5, ref_frame=40)
video_norm = om.video.normalize_pixelwise_slidingwindow(video_warped, window_size=60)

Because the rabbit heart was stained with the voltage-sensitive dye Di-4-ANEPPS, the tissue becomes darkens when the tisue depolarizes:

In [ ]:
om.video.play2(video, video_norm,
               title1="original video",
               title2="warped, normalized video");

In [ ]:
render_ani_func(lambda: om.video.play2(video, video_norm, title1="original video", title2="warped, normalized video", interval=20))

We can now compute an activation map by identifying the timepoints in each pixel that correspond to when the action potential wave front passes through that pixel.

## Computing Activation Maps from Pixel-wise Normalized Optical Maps

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

In [ ]:
#video_diff = om.video.temporal_difference(video_warped, 5)
#video_diff[:, background_mask] = np.nan